In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time

import torch
import torch.optim as optim
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader

from dataloader import *
from models import *

In [2]:
def rand(random_seed = 777):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

In [3]:
import random

In [4]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")

dataset_dir = "./Datasets/"
batch_size = 8
num_epochs = 20
lr = 0.001

save_dir = './CheckPoint/'

num_workers = 0
transform = transforms.Compose([
        transforms.ColorJitter(brightness = 1),
        transforms.ToTensor()
])

transform_mask = transforms.Compose([
        transforms.ToTensor(),
])

transform_test = transforms.Compose([
        transforms.Resize((1024,1024)),
        transforms.ToTensor()
])

train_dataset = FaceMask(dataset_dir, transform, transform_mask)
train_dataloader = DataLoader(train_dataset, batch_size = batch_size,
                             shuffle = True, num_workers = num_workers)

test_data = FaceMask(dataset_dir, transform, transform_test = transform_test, test = True)
test_dataloader = DataLoader(test_data, batch_size = batch_size,
                            shuffle = False, num_workers = num_workers)
sample = next(iter(test_dataloader))
test_img = sample['test_img'].to(device)
show_img = sample['show_img']

In [5]:
def save_checkpoint(state, dirpath, epoch):
    filename = 'checkpoint-{}.pt'.format(epoch)
    checkpoint_path = os.path.join(dirpath, filename)
    torch.save(state, checkpoint_path)
    print('--- checkpoint saved to ' + str(checkpoint_path) + ' ---')

In [6]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        nn.utils.spectral_norm(m)
        
    if classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [7]:
model = Unet().to(device)
model.apply(weights_init)

optimizer = optim.Adam(model.parameters(), lr = lr, betas = (0.5,0.999))

In [8]:
dataloader = train_dataloader
model.train()
loss = nn.BCELoss()
Start = time.time()

Loss_list = []
for epoch in range(num_epochs):
    print("Epoch {}/{}".format(epoch+1, num_epochs))
    print("-" * 10)
    start = time.time()
    
    face_masked = None
    mask = None
    for i, sample in enumerate(dataloader, 0):
        face_masked, mask = sample['W_mask'], sample['mask']
        batch_size = mask.size(0)
        
        face_masked = face_masked.to(device)
        mask = mask.to(device)
        
        optimizer.zero_grad()
        
        output = model(face_masked)
        Loss = loss(output, mask)
        Loss_list.append(Loss.item())
        Loss.backward()
        optimizer.step()
        
        if (i % 600 == 0):
            print("[{:d}/{:d}] Loss : {:.4f}".format(i, len(dataloader), Loss.item()))
    
    save_checkpoint({
        'epoch' : epoch + 1,
        'model_state_dict' : model.state_dict(),
        'model_opt' : optimizer.state_dict()
    }, save_dir, epoch + 1)
    
    print("=" * 100)
    print('Time taken by epoch: {:.0f}h {:.0f}m {:.0f}s'.format(((time.time() - start) // 60) // 60, (time.time() - start) // 60, (time.time() - start) % 60))
    print()
    
    with torch.no_grad():
        result = model(face_masked).cpu()
        test_result = model(test_img).cpu()
        m = mask.cpu()
        sample = []
        test_sample = []
        for i in range(batch_size):
            sample.extend([m[i], result[i]])
            test_sample.append(test_result[i])
        
        result_img = utils.make_grid(sample, padding = 2,
                                        normalize = True, nrow = 2)
        test_result_img = utils.make_grid(test_sample, padding = 0,
                                        normalize = True, nrow = 1)
    
        utils.save_image(result_img, "./result/result-{}epoch.png".format(epoch + 1))
        utils.save_image(test_result_img, "./result/test_result-{}epoch.png".format(epoch + 1))
        
print("Training is finished")
hour = ((time.time() - Start) // 60) // 60
print('Time taken by num_epochs: {:.0f}h {:.0f}m {:.0f}s'.format(hour, (time.time() - Start) - hour * 60, (time.time() - Start) % 60))

Epoch 1/20
----------
[0/1250] Loss : 0.6665
[600/1250] Loss : 0.1480
--- checkpoint saved to ./CheckPoint/checkpoint-1.pt ---
Time taken by epoch: 0h 33m 44s

Epoch 2/20
----------
[0/1250] Loss : 0.2455
[600/1250] Loss : 0.0569
[1200/1250] Loss : 0.0171
--- checkpoint saved to ./CheckPoint/checkpoint-2.pt ---
Time taken by epoch: 0h 33m 35s

Epoch 3/20
----------
[0/1250] Loss : 0.0149
[600/1250] Loss : 0.0286
[1200/1250] Loss : 0.0471
--- checkpoint saved to ./CheckPoint/checkpoint-3.pt ---
Time taken by epoch: 0h 33m 43s

Epoch 4/20
----------
[0/1250] Loss : 0.0081
[600/1250] Loss : 0.0588
[1200/1250] Loss : 0.0119
--- checkpoint saved to ./CheckPoint/checkpoint-4.pt ---
Time taken by epoch: 0h 33m 38s

Epoch 5/20
----------
[0/1250] Loss : 0.0111
[600/1250] Loss : 0.0336
[1200/1250] Loss : 0.0122
--- checkpoint saved to ./CheckPoint/checkpoint-5.pt ---
Time taken by epoch: 0h 33m 37s

Epoch 6/20
----------
[0/1250] Loss : 0.0129
[600/1250] Loss : 0.0540
[1200/1250] Loss : 0.0138
